In [1]:
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.utils import vis_utils as vizu
from keras.layers import Dropout
from keras import callbacks
from keras import backend
from scipy.integrate import simps
from scipy.stats import f
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, mutual_info_regression
def knn_baseline_model():
    
    def root_mean_squared_error(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
    # create model
    model = Sequential()
    #model.add(GaussianNoise(stddev=0.1, input_shape=(14,)))
    model.add(Dense(7, activation='relu', input_shape=(15,)))
    model.add(Dropout(0.5))
    model.add(GaussianNoise(0.1))
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=root_mean_squared_error, optimizer='adam')
    #model.compile(loss='mse', optimizer='adam')
    return model
def coeff_determination(y_true, y_pred):
    ssres = np.sum(np.square(y_true - y_pred))
    sstot = np.sum(np.square(y_true - np.mean(y_pred)))
    return 1 - ssres / sstot
def REC(y_true , y_pred):
    
    Accuracy = []
    Begin_Range = 0
    End_Range = 1.5
    Interval_Size = 0.01
    
    # List of epsilons
    Epsilon = np.arange(Begin_Range , End_Range , Interval_Size)
    
    # Main Loops
    for i in range(len(Epsilon)):
        count = 0.0
        for j in range(len(y_true)):
            y_true[j]=float(y_true[j])
            y_true[j]
            y_pred[j]
            
            np.linalg.norm(y_true[j] - y_pred[j])
            np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]
            
            if np.linalg.norm(y_true[j] - y_pred[j]) / np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]:
                count = count + 1
        
        Accuracy.append(count/len(y_true))
    
    # Calculating Area Under Curve using Simpson's rule
    AUC = simps(Accuracy , Epsilon ) / End_Range
        
    # returning epsilon , accuracy , area under curve    
    return Epsilon, Accuracy, AUC
def model_evaluate (model, X_train, X_test, y_train, y_test):

    y_pred = model.predict(X_test)
    Deviation, Accuracy, auc = REC(y_test.values, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    #r2 = r2_score(y_test, y_pred)
    r2 = coeff_determination(y_test, y_pred)
    mape = ((y_test - y_pred) / y_test).abs().mean()*100
    
    f_test = np.var(y_test) / np.var(y_pred)
    df1 = len(y_test) - 1
    df2 = len(y_pred) - 1
    p_value = 1 - 2 * abs(0.5 - f.cdf(f_test, df1, df2))
    
    p_stars = ''
    if p_value <= 0.05:
        p_stars = '*'
    if p_value <= 0.01:
        p_stars = '**'
    if p_value <= 0.001:
        p_stars = '***'
    #print(F, p_value)
    
    y_validate = model.predict(X_train)
    training_loss = sqrt(mean_squared_error(y_train, y_validate))
    
    return_dict = {'rmse':rmse, 
                   'mape':mape, 
                   'r2':r2, 
                   'auc':auc,
                   'training_loss':training_loss, 
                   'f_test':f_test,
                   'p':p_stars,
                   'y_pred':y_pred, 
                   'y_validate':y_validate}
    
    return return_dict
'''
conn = MongoClient('140.115.53.147', 27017)
db = conn['KU_dataset_1']
collection = db['data1_15features.csv']
all_features = pd.DataFrame(list(collection.find()))
all_features = all_features.drop(['_id'], axis=1)
collection = db['data1_score.csv']
final_score = pd.DataFrame(list(collection.find()))
final_score = final_score.drop(['_id'], axis=1)
'''


C:\Users\Liuya\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


"\nconn = MongoClient('140.115.53.147', 27017)\ndb = conn['KU_dataset_1']\ncollection = db['data1_15features.csv']\nall_features = pd.DataFrame(list(collection.find()))\nall_features = all_features.drop(['_id'], axis=1)\ncollection = db['data1_score.csv']\nfinal_score = pd.DataFrame(list(collection.find()))\nfinal_score = final_score.drop(['_id'], axis=1)\n"

In [2]:
KU = pd.read_csv('KU_dataset_1.csv')
KU.fillna(0.0, inplace=True)
KU = KU.drop(['Userid'], axis=1)
KU.head(3)

,Add_Bookmark,Add_Marker,Add_Memo,Change_Memo,Close,Delete_Bookmark,Delete_Marker,Delete_Memo,Jump,Next,Open,Prev,Search,Marker,Memo,Score
0,0,0,0,0,3,0,0,0,65,1070,27,141,0,0,0,80
1,0,0,0,0,0,0,0,0,0,5,1,2,0,0,0,80
2,1,2,0,0,1,1,0,0,1,296,5,147,0,2,0,60


In [3]:
data=KU
data.to_csv('KU_data1.csv')
data_y= data['Score']
data_x = data.drop(['Score'], axis=1)

In [4]:
data

,Add_Bookmark,Add_Marker,Add_Memo,Change_Memo,Close,Delete_Bookmark,Delete_Marker,Delete_Memo,Jump,Next,Open,Prev,Search,Marker,Memo,Score
0,0,0,0,0,3,0,0,0,65,1070,27,141,0,0,0,80
1,0,0,0,0,0,0,0,0,0,5,1,2,0,0,0,80
2,1,2,0,0,1,1,0,0,1,296,5,147,0,2,0,60
3,1,1,1,0,0,1,1,0,2,48,4,29,0,2,2,50
4,0,0,0,0,3,0,0,0,2,459,5,71,0,0,0,100
5,0,0,0,0,0,0,0,0,3,179,2,74,9,0,0,85
6,0,0,0,0,3,0,0,0,0,499,6,110,0,0,0,90
7,0,0,0,0,0,0,0,0,0,1186,8,756,0,0,0,100
8,0,0,0,0,3,0,0,0,4,278,10,66,0,0,0,60
9,0,0,0,0,6,0,0,0,2,766,10,79,0,0,0,75


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

In [6]:
mlp = KerasRegressor(build_fn=knn_baseline_model, epochs=1000, batch_size=2, verbose=0)

In [7]:
mlp.fit(X_train, y_train) 
result_dict = model_evaluate (mlp, X_train, X_test, y_train, y_test)
print(result_dict)

{'rmse': 24.14707726955135, 'mape': 38.04731252318934, 'r2': -0.0674802363582534, 'auc': 0.8616666666666667, 'training_loss': 23.687114089087558, 'f_test': 56.31454199373607, 'p': '***', 'y_pred': array([72.51372 , 72.31295 , 75.06511 , 74.88783 , 75.54352 , 74.60373 ,
       74.50218 , 73.71347 , 75.108444, 85.75366 , 75.635956, 74.37822 ,
       77.15018 , 72.78235 , 73.744446, 73.23755 ], dtype=float32), 'y_validate': array([87.24849 , 80.222946, 74.109314, 78.869934, 79.780914, 79.637054,
       74.06464 , 73.97896 , 76.11119 , 72.30943 , 73.99686 , 84.74304 ,
       72.6871  , 77.06143 , 79.319244, 81.4305  , 72.31352 , 78.698044,
       81.8625  , 72.30689 , 72.94339 , 73.62897 , 72.3064  , 72.877014,
       73.20134 , 75.95094 , 79.68767 , 79.93787 , 75.24875 , 72.310196,
       81.13106 , 75.38257 , 73.46942 , 74.86348 , 81.32396 , 72.77077 ,
       72.31092 ], dtype=float32)}
